In [42]:
import pandas as pd 
import numpy as np
import datetime
from datetime import date

# New Data

In [6]:
info_url ='https://github.com/owid/covid-19-data/raw/master/public/data/owid-covid-data.csv'

In [7]:
df = pd.read_csv(info_url)

In [8]:
df.isna().sum()

iso_code                                        0
continent                                    9072
location                                        0
date                                            0
total_cases                                  2609
                                            ...  
human_development_index                     26951
excess_mortality_cumulative_absolute       145189
excess_mortality_cumulative                145189
excess_mortality                           145189
excess_mortality_cumulative_per_million    145189
Length: 67, dtype: int64

In [9]:
continents_location = df[df['continent'].isna()]['location'].unique()

In [10]:
for x in continents_location:
    df = df.drop(df[df.location==x].index)

In [11]:
df.isna().sum()

iso_code                                        0
continent                                       0
location                                        0
date                                            0
total_cases                                  2602
                                            ...  
human_development_index                     18585
excess_mortality_cumulative_absolute       136117
excess_mortality_cumulative                136117
excess_mortality                           136117
excess_mortality_cumulative_per_million    136117
Length: 67, dtype: int64

In [12]:
df = df.fillna(0)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141323 entries, 0 to 150394
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    141323 non-null  object 
 1   continent                                   141323 non-null  object 
 2   location                                    141323 non-null  object 
 3   date                                        141323 non-null  object 
 4   total_cases                                 141323 non-null  float64
 5   new_cases                                   141323 non-null  float64
 6   new_cases_smoothed                          141323 non-null  float64
 7   total_deaths                                141323 non-null  float64
 8   new_deaths                                  141323 non-null  float64
 9   new_deaths_smoothed                         141323 non-null  float64
 

In [14]:
df.select_dtypes(object).sample(5)

,iso_code,continent,location,date,tests_units
25472,CPV,Africa,Cape Verde,2020-09-08,tests performed
105639,PSE,Asia,Palestine,2021-09-09,tests performed
112467,RUS,Europe,Russia,2021-08-16,tests performed
30612,COG,Africa,Congo,2020-05-12,0
1396,ALB,Europe,Albania,2020-04-04,tests performed


In [15]:
drop_cols=['iso_code','tests_units']

In [16]:
df = df.drop(columns=drop_cols)

In [17]:
df.sample(5)

,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
125329,Africa,South Africa,2021-11-01,2922222.0,106.0,349.143,89179.0,2.0,35.0,48669.635,...,8.1,33.2,43.993,2.32,64.13,0.709,0.0,0.0,0.0,0.0
24234,Africa,Cameroon,2020-12-29,26277.0,0.0,61.143,448.0,0.0,0.0,965.205,...,0.0,0.0,2.735,1.30,59.29,0.563,0.0,0.0,0.0,0.0
145157,Oceania,Vanuatu,2021-07-30,4.0,0.0,0.000,1.0,0.0,0.0,12.720,...,2.8,34.5,25.209,0.00,70.47,0.609,0.0,0.0,0.0,0.0
85048,Asia,Maldives,2020-04-08,19.0,0.0,0.000,0.0,0.0,0.0,34.951,...,2.1,55.0,95.803,0.00,78.92,0.740,0.0,0.0,0.0,0.0
95953,Asia,Nepal,2020-05-12,217.0,83.0,19.286,0.0,0.0,0.0,7.313,...,9.5,37.8,47.782,0.30,70.78,0.602,0.0,0.0,0.0,0.0


In [18]:
for x in df.select_dtypes(float).columns:
    df[x] = df[x].astype(int)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141323 entries, 0 to 150394
Data columns (total 65 columns):
 #   Column                                      Non-Null Count   Dtype 
---  ------                                      --------------   ----- 
 0   continent                                   141323 non-null  object
 1   location                                    141323 non-null  object
 2   date                                        141323 non-null  object
 3   total_cases                                 141323 non-null  int64 
 4   new_cases                                   141323 non-null  int64 
 5   new_cases_smoothed                          141323 non-null  int64 
 6   total_deaths                                141323 non-null  int64 
 7   new_deaths                                  141323 non-null  int64 
 8   new_deaths_smoothed                         141323 non-null  int64 
 9   total_cases_per_million                     141323 non-null  int64 
 10  new_case

In [20]:
df['date'] = pd.to_datetime(df['date'])

In [21]:
df.sample(5)

,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
4788,North America,Antigua and Barbuda,2020-07-24,82,6,0,3,0,0,830,...,0,0,0,3,77,0,0,0,0,0
99810,Africa,Niger,2021-12-05,7080,19,17,261,0,1,281,...,0,15,8,0,62,0,0,0,0,0
47362,South America,Falkland Islands,2021-12-10,83,0,0,0,0,0,23526,...,0,0,0,0,81,0,0,0,0,0
58858,South America,Guyana,2021-06-07,17718,53,109,411,2,3,22418,...,0,0,77,1,69,0,0,0,0,0
97612,Oceania,New Caledonia,2021-04-17,123,2,0,0,0,0,426,...,0,0,0,0,77,0,0,0,0,0


In [22]:
df.head()

,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,Asia,Afghanistan,2020-02-24,5,5,0,0,0,0,0,...,0,0,37,0,64,0,0,0,0,0
1,Asia,Afghanistan,2020-02-25,5,0,0,0,0,0,0,...,0,0,37,0,64,0,0,0,0,0
2,Asia,Afghanistan,2020-02-26,5,0,0,0,0,0,0,...,0,0,37,0,64,0,0,0,0,0
3,Asia,Afghanistan,2020-02-27,5,0,0,0,0,0,0,...,0,0,37,0,64,0,0,0,0,0
4,Asia,Afghanistan,2020-02-28,5,0,0,0,0,0,0,...,0,0,37,0,64,0,0,0,0,0


In [23]:
def get_col(df,column):
    ## Get the subset df to work off
    cases = df[['location','date',column]]

    ## Create the dates variable and sort it
    dates = cases['date'].unique()
    dates.sort()


    ## Get the len of dates
    len_dates = len(dates)

    ## Get the unique locations
    locations = [x for x in cases['location'].unique()]

    data = list()
    ## Loop through each location to get the column and merge it to the dataframe 
    for x in locations:
        y = ()
        len_y = 0 
        y = list(cases[cases['location']==x][column])
        len_y = len(y)
        len_zeroes = len_dates-len_y
        zeroes = list(np.zeros(len_zeroes))
        merge = zeroes + y
        data.append(merge)
    data = np.array(data).T
    df2 = pd.DataFrame(data,columns=locations,index=dates)
    df2 = df2.astype(int)
    
    return df2

In [24]:
deaths = get_col(df,'total_deaths')

In [25]:
cases = get_col(df,'total_cases')

In [26]:
new_cases = get_col(df,'new_cases')

In [27]:
new_deaths = get_col(df,'new_deaths')

In [28]:
def get_world(col):
    date = df['date'].max()

    continent_cases = df[df['date']==date.strftime('%Y-%m-%d')]

    total_cases_world = pd.DataFrame(continent_cases.groupby('continent')[col].sum())
    
    return total_cases_world

In [29]:
total_cases_world = get_world('total_cases')

In [30]:
new_cases_world = get_world('new_cases')

# Vaccination Data

In [31]:
vac_url = 'https://github.com/owid/covid-19-data/raw/master/public/data/vaccinations/vaccinations.csv'

In [32]:
vac_df = pd.read_csv(vac_url)

In [33]:
vac_df.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0,1367.0,0.003


In [34]:
vac_df_max = pd.DataFrame(columns=vac_df.columns)
for location in vac_df['location'].unique():
    result = vac_df.loc[(vac_df['location']==location)&(vac_df['date']==vac_df.loc[(vac_df['location']==location)]['date'].max())]
    vac_df_max = pd.concat((vac_df_max,result))

In [35]:
for x in continents_location:
    vac_df_max = vac_df_max.drop(vac_df_max[vac_df_max.location==x].index)

In [36]:
vac_df_max = vac_df_max.fillna(0)
vac_df_max['date'] = pd.to_datetime(vac_df_max['date'])
date = vac_df_max['date'].max()

In [37]:
vac_cols = [x for x in vac_df_max.columns[7::]]
vac_cols.append('iso_code')
vac_cols.append('date')
vac_cols.append('total_vaccinations')
vac_df_max = vac_df_max.drop(columns=vac_cols)
vac_df_max = vac_df_max.set_index('location').T

In [38]:
vac_df_max = vac_df_max.astype(int)
vac_df_max = vac_df_max.rename(index={'people_vaccinated': "People Vaccinated", 
                     'people_fully_vaccinated': "People Fully Vaccinated", 
                     'total_boosters': "Total Boosters"})

# Dash

In [39]:
import dash
from dash import dash_table
from dash import html as html
from dash import dcc as dcc
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
from dash import State, html
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash

In [40]:
button_group = dbc.ButtonGroup(
    [dbc.Button("Learn More",href='https://www.who.int/emergencies/diseases/novel-coronavirus-2019'), dbc.Button("Github",href='https://github.com/ronanmccormack-ca/DAV2021-Mini-Project')]
)

In [41]:
app = dash.Dash(__name__,external_stylesheets=[dbc.themes.SKETCHY])
server = app.server

loc_columns = [x for x in df['location'].unique()]
night_colors = ['rgb(56, 75, 126)', 'rgb(18, 36, 37)', 'rgb(34, 53, 101)',
                'rgb(36, 55, 57)', 'rgb(6, 4, 4)']
irises_colors = ['rgb(33, 75, 99)', 'rgb(79, 129, 102)', 'rgb(151, 179, 100)',
                 'rgb(175, 49, 35)', 'rgb(36, 73, 147)']
top_cases_country = [x for x in df[df['date']==df['date'].max()].sort_values(by='total_cases',ascending=False)['location'].head(5)]

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'COVID-19 Dashboard', style = {'textAlign':'center',\
                                            'marginTop':30,'marginBottom':30}),
    html.Div(children='''
        Created by Ronan McCormack S00144576
    ''',style={
            'textAlign': 'center',
            'color': 'primary',
            'marginBottom':10
        }),
    html.Div(id = 'cases_graph', children= [
        dbc.Row([ 
            button_group
        ]),
        dcc.Dropdown(
                id='dropdown',
                options=[{'label': i, 'value': i} for i in loc_columns],
                value='Ireland',
                style={
                    'width':'90%',
                    'padding-left':'20%',
                    'marginTop':40
                }
            ),
        ]),
    html.Div(id='new_case_graph',children=[
        dcc.Graph(id='cases_plot',style={'display': 'inline-block', 'width': '50%'}),
        dcc.Graph(id='new_case_plot',style={'display': 'inline-block', 'width': '50%'})
    ]),
    dbc.Container([    
    dcc.Graph(id='top_country_plot',style={'display': 'inline-block', 'width': '99%'}),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i} for i in top_cases_country],
        data=cases.iloc[-1:].to_dict('records'),
    ),
]),
    html.Div(id='vac_graph',children=[
         dcc.Graph(id='new_world_plot',style={'display': 'inline-block', 'width': '50%'}),
        dcc.Graph(id='total_world_plot',style={'display': 'inline-block', 'width': '50%'})
    ]),
])

## App Callbacks
@app.callback(Output(component_id='cases_plot', component_property= 'figure'),
              (Output(component_id='new_case_plot',component_property='figure')),
              (Output(component_id='total_world_plot',component_property='figure')),
              (Output(component_id='new_world_plot',component_property='figure')),
              (Output(component_id='top_country_plot',component_property='figure')),
              [Input(component_id='dropdown', component_property= 'value')])
              
def cases_update(dropdown_value):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = cases.index, y = cases['{}'.format(dropdown_value)],\
                      mode='lines',name='Total Cases',line = dict(color = 'black', width = 4)))
    fig.add_trace(go.Scatter(x = deaths.index, y = deaths['{}'.format(dropdown_value)],\
                     mode='lines',name='Total Deaths',line = dict(color = 'red', width = 4)))
    fig.update_layout(title={'text':'Total Cases & Deaths', 
                      'y':0.9,'x':0.5,'yanchor': 'top'},
                      xaxis_title = 'Date',
                      yaxis_title = 'Confirmed Cases & Deaths'
                      )
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(x = new_cases.index, y = new_cases['{}'.format(dropdown_value)],\
                     name='New Cases',marker = dict(color = 'black')))
    fig2.add_trace(go.Bar(x = new_deaths.index, y = new_deaths['{}'.format(dropdown_value)],\
                     name='New Deaths',marker = dict(color = 'red')))
    fig2.update_layout(title={'text':'New Cases & Deaths',           
                       'y':0.9,'x':0.5,'yanchor': 'top'},
                      xaxis_title = 'Date',
                      yaxis_title = 'Confirmed New Cases',
                      )
    fig4 = go.Figure([go.Bar(x=total_cases_world.index, y=total_cases_world['total_cases'],
                            marker_color=irises_colors)
                     ])

    fig4.update_layout(title={'text':'Total COVID Cases Per Continent'}
                      )
    fig5 = go.Figure([go.Pie(labels=vac_df_max.index, values=vac_df_max['{}'.format(dropdown_value)],
                            hole=.3,marker_colors=night_colors)
                     ])

    fig5.update_layout(title={'text':'Percentage of All People Vaccinationed in '+'{}'.format(dropdown_value)}
                      )
    top_cases_country.append('{}'.format(dropdown_value))
    fig6 = go.Figure()
    for country in top_cases_country:
        fig6.add_trace(go.Scatter(x = cases.index, y = cases[country],\
                                 name=country,))
    
    fig6.update_layout(title={'text':'{}'.format(dropdown_value)+' Compared to Top 5 Countries - Total Cases',           
                   'y':0.9,'x':0.5,'yanchor': 'top'},
                  xaxis_title = 'Date',
                  yaxis_title = 'Confirmed New Cases',
                  )

    return fig, fig2, fig4, fig5, fig6
if __name__ == '__main__': 
    app.run_server(host='localhost',port=8000)

Dash is running on http://localhost:8000/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8000/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [28/Dec/2021 19:22:47] "GET /_dash-component-suites/d